# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [7]:
document = ET.parse( './data/mondial_database.xml' )

## 1: 10 countries with the lowest infant mortality rates

In [32]:
import numpy as np
import pandas as pd

In [47]:
countries = []
child_mortality_rates = []

for child in document.findall('country'):
    countries.append(child.find('name').text)
    try:
        child_mortality_rates.append(float(child.find('infant_mortality').text))
    except:
        child_mortality_rates.append(np.nan)

print len(countries), len(child_mortality_rates)

244 244


### 1. Answer:

In [48]:
infant_mortality_df = pd.DataFrame({'country': countries,
                                    'child_mortality': child_mortality_rates})
infant_mortality_df.sort_values(by='child_mortality', ascending=False).head(10)

,child_mortality,country
194,145.82,Western Sahara
54,117.23,Afghanistan
189,104.34,Mali
226,100.14,Somalia
213,92.86,Central African Republic
230,90.92,Guinea-Bissau
214,90.30,Chad
192,86.27,Niger
195,79.99,Angola
201,76.80,Burkina Faso


## 2: 10 cities with the largest population

In [77]:
cities = []
populations = []

for city in document.iter('city'):
    cities.append(city.find('name').text)
    try:
        populations.append(int(city.findall('population')[-1].text))
    except:
        populations.append(np.nan)

print len(cities), len(populations)

3380 3380


### 2. Answer:

In [78]:
city_populations_df = pd.DataFrame({'city': cities,
                                    'population': populations})
city_populations_df.sort_values(by='population', ascending=False).head(10)

,city,population
1341,Shanghai,22315474.0
771,Istanbul,13710512.0
1527,Mumbai,12442373.0
479,Moskva,11979529.0
1340,Beijing,11716620.0
2810,São Paulo,11152344.0
1342,Tianjin,11090314.0
1064,Guangzhou,11071424.0
1582,Delhi,11034555.0
1067,Shenzhen,10358381.0


## 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [99]:
countries2 = []
ethnic_groups = []
country_populations = []
ethnic_populations = []
percentages = []

for country in document.iter('country'):
    population = int(country.findall('population')[-1].text)
    groups = country.findall('ethnicgroup')
    for group in groups:
        ethnic_population = int(population * float(group.attrib['percentage'])/100)
        countries2.append(country.find('name').text)
        ethnic_groups.append(group.text)
        percentages.append(float(group.attrib['percentage'])/100)
        ethnic_populations.append(ethnic_population)
        country_populations.append(population)


print len(countries2),len(ethnic_groups), len(populations), len(percentages)

628 628 628 628


### 3. Answer:

In [107]:
ethnic_groups_df = pd.DataFrame({'ethnic_group': ethnic_groups,
                                 'ethnic_population': ethnic_populations})
ethnic_groups_df['ethnic_population'].groupby(ethnic_groups_df['ethnic_group']).sum().sort_values(ascending=False).head(10)

ethnic_group
Han Chinese    1245058800
Indo-Aryan      871815583
European        494872201
African         318325104
Dravidian       302713744
Mestizo         157734349
Bengali         146776916
Russian         131856989
Japanese        126534212
Malay           121993548
Name: ethnic_population, dtype: int64

## 4a: Name and country of longest river, b) largest lake and c) airport at highest elevation

In [117]:
car_codes = []

for country in document.findall('country'):
    car_codes.append(country.attrib['car_code'])
    
car_code_df = pd.DataFrame({'country': countries,
                            'car_code': car_codes})

In [149]:
rivers = []
lengths = []
country_code = []

for river in document.iter('river'):   
    car_codes = river.attrib['country'].split()
    for car_code in car_codes:
        rivers.append(river.find('name').text) 
        country_code.append(car_code)
        try:
            lengths.append(float(river.find('length').text))
        except:
            lengths.append(np.nan)

In [150]:
river_df = pd.DataFrame({'river': rivers,
                         'length': lengths,
                         'car_code': country_code})
river_df = river_df.merge(car_code_df).sort_values(by=['length', 'country'], ascending=[False, True])

### 4a. Answer:

In [151]:
river_df[['river', 'country']][(river_df['length'] == river_df['length'].max())]

,river,country
302,Amazonas,Brazil
299,Amazonas,Colombia
310,Amazonas,Peru


## 4b: Name and country of largest lake

In [155]:
lakes = []
lake_car_codes = []
areas = []

for lake in document.iter('lake'):   
    car_codes = lake.attrib['country'].split()
    for car_code in car_codes:
        lakes.append(lake.find('name').text) 
        lake_car_codes.append(car_code)
        try:
            areas.append(float(lake.find('area').text))
        except:
            areas.append(np.nan)
            

In [158]:
lakes_df = pd.DataFrame({'lake': lakes,
                         'area': areas,
                         'car_code': lake_car_codes})
lakes_df = lakes_df.merge(car_code_df).sort_values(by=['area', 'country'], ascending=[False, True])

### 4b. Answer:

In [161]:
lakes_df[['lake', 'country']][(lakes_df['area'] == lakes_df['area'].max())]

,lake,country
73,Caspian Sea,Azerbaijan
72,Caspian Sea,Iran
74,Caspian Sea,Kazakhstan
56,Caspian Sea,Russia
77,Caspian Sea,Turkmenistan


## 4c: Name and country of airport at highest elevation

In [166]:
airports = []
airport_car_codes = []
elevations = []

for airport in document.iter('airport'):   
    airports.append(airport.find('name').text) 
    airport_car_codes.append(airport.attrib['country'])
    try:
        elevations.append(float(airport.find('elevation').text))
    except:
        elevations.append(np.nan)

In [168]:
airports_df = pd.DataFrame({'airport': airports,
                            'car_code': airport_car_codes,
                            'elevation': elevations})
airports_df = airports_df.merge(car_code_df).sort_values(by='elevation', ascending=False)


### 4c. Answer:

In [169]:
airports_df[['airport', 'country']][(airports_df['elevation'] == airports_df['elevation'].max())]

,airport,country
80,El Alto Intl,Bolivia
